In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms

In [2]:
import sys
sys.path.insert(1, "../../")

In [3]:
from data_preprocessing import get_means, get_stds, ImageNetSubset
from Models.darknet19 import Darknet19

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [5]:
data_path = '../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [6]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [7]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [8]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

# Inference

In [24]:
model = Darknet19(num_classes=1000, device=device, dtype=torch.float32)

In [25]:
img = train_dataset[0][0].unsqueeze(0)

In [26]:
from torch.profiler import profile, ProfilerActivity

In [27]:
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    # schedule=torch.profiler.schedule(wait=0, warmup=0, active=1, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../log/darknet19/inference'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=True) as prof:
   out = model(img)
print(prof.key_averages().table(row_limit=-1))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total GFLOPs  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Conv block         7.39%       9.074ms        80.42%      98.734ms       5.485ms      51.73 Mb           0 b            18            --  
                     aten::conv2d         0.22%     267.500us        55.62%      68.292ms       3.594ms      17.41 Mb           0 b            19         5.582  
                aten::convolution         0.54%     660.400us        55.40%      68.024ms       3.580ms      17.41 Mb           0 b            19            --  
               aten::_convol